# Model Testing


## Setup


In [ ]:
from scripts.data_loader import load_data, split_features_targets
from scripts.preprocessing import preprocess_data, create_features
from scripts.model_testing import test_independant_models, find_top_models
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RANSACRegressor,
    TheilSenRegressor,
    HuberRegressor,
)
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    ExtraTreesRegressor,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression
from sklearn.ensemble import BaggingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
df = load_data("data")
df = preprocess_data(df)
# df = create_features(df)

features, targets = split_features_targets(df)

## Initial Testing


### Models


In [ ]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1),
    AdaBoostRegressor(n_estimators=50, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    DecisionTreeRegressor(max_depth=10),
    KNeighborsRegressor(n_neighbors=5, n_jobs=-1),
    MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500),
    SVR(kernel="linear", C=1.0, epsilon=0.1),
    LinearSVR(C=1.0, epsilon=0.1, max_iter=1000, tol=1e-3),
    NuSVR(kernel="linear", C=1.0, nu=0.5),
    GaussianProcessRegressor(n_restarts_optimizer=0, normalize_y=True),
    PLSRegression(n_components=6),
    KernelRidge(alpha=1.0, kernel="linear"),
    TheilSenRegressor(max_subpopulation=1e4, n_subsamples=None),
    HuberRegressor(epsilon=1.35, max_iter=1000),
    BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1),
    VotingRegressor(
        estimators=[
            ("lr", LinearRegression()),
            ("rf", RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)),
            (
                "gbr",
                GradientBoostingRegressor(
                    n_estimators=50, max_depth=3, learning_rate=0.1
                ),
            ),
        ]
    ),
    StackingRegressor(
        estimators=[
            ("lr", LinearRegression()),
            ("rf", RandomForestRegressor(n_estimators=50, max_depth=5, n_jobs=-1)),
            (
                "gbr",
                GradientBoostingRegressor(
                    n_estimators=50, max_depth=3, learning_rate=0.1
                ),
            ),
        ]
    ),
    XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1),
    LGBMRegressor(
        n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1, verbose=-1
    ),
    CatBoostRegressor(
        iterations=100, depth=5, learning_rate=0.1, thread_count=-1, verbose=0
    ),
]

### Results


In [ ]:
# Ice Thickness
find_top_models(
    test_independant_models(models, features, targets["ice_thickness"], 10000, 0.2)
).head()

In [ ]:
# Ice Velocity
find_top_models(
    test_independant_models(models, features, targets["ice_velocity"], 10000, 0.2)
).head()

In [ ]:
# Ice Mask
find_top_models(
    test_independant_models(models, features, targets["ice_mask"], 10000, 0.2)
).head()

## Further Testing


### Models


In [ ]:
models = [
    RandomForestRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1),
    ExtraTreesRegressor(n_estimators=100, max_depth=10, n_jobs=-1),
    # SVR(kernel="linear", C=1.0, epsilon=0.1),
    # LinearSVR(C=1.0, epsilon=0.1, max_iter=1000, tol=1e-3),
    # NuSVR(kernel="linear", C=1.0, nu=0.5),
    # GaussianProcessRegressor(n_restarts_optimizer=0, normalize_y=True),
    BaggingRegressor(n_estimators=10, max_samples=0.5, n_jobs=-1),
    XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1),
    LGBMRegressor(
        n_estimators=100, max_depth=5, learning_rate=0.1, n_jobs=-1, verbose=-1
    ),
]

### Results


In [ ]:
results = test_independant_models(models, features, targets["ice_velocity"], 10000, 0.2)
results.head(10)